### Environment Setup and Dependencies

Importing necessary libraries for document loading, text splitting, vector embeddings, retrieval-augmented generation (RAG), and conversational memory. Configuring the environment to suppress warnings and prepare for processing the PDF document with LangChain and Google Generative AI.

In [ ]:
!pip install -U langchain-community langchain-google-genai pypdf langchain_huggingface chromadb arabic-reshaper python-bidi unstructured pdfminer.six Pillow pi_heif "unstructured[pdf]" sentence-transformers deep_translator

In [2]:
import os
import time
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from langchain.chains import create_history_aware_retriever, create_retrieval_chain, RetrievalQA
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
import arabic_reshaper
from bidi.algorithm import get_display

import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
print("Using device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Using device: cpu


### Load PDF Document

Loading the internal regulations document of the Faculty of Computers and Artificial Intelligence (October 2019) using PyPDFLoader.

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahmedayman7/fcai-usc-internal-regulations")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fcai-usc-internal-regulations' dataset.
Path to dataset files: /kaggle/input/fcai-usc-internal-regulations


In [5]:
pdf_path = "/kaggle/input/fcai-usc-internal-regulations/2019.pdf"

loader = PyPDFLoader(pdf_path, mode="page")
docs = loader.load()

# Each doc corresponds to a page
print(len(docs), "pages loaded")

74 pages loaded


### Fix Arabic Text Rendering

Reshaping and reordering Arabic text in each document page to ensure proper visual display (connected characters and right-to-left layout). The corrected text is saved back into the `Document` objects.  
Then, previewing the first three pages to verify the fix.

In [7]:
# Fix Arabic text inside each Document object
for i, doc in enumerate(docs):
    # Extract raw text from the page
    text = doc.page_content

    # Reshape Arabic characters to connect properly
    reshaped_text = arabic_reshaper.reshape(text)

    # Correct the right-to-left display order
    bidi_text = get_display(reshaped_text)

    # Save the fixed Arabic text back into the Document
    doc.page_content = bidi_text

print("✅ Arabic text has been reshaped, reordered, and stored back into docs.\n")

# Display the content of the first few pages (example: first 3)
for i, page in enumerate(docs[:3]):
    print(f"--- Page {i + 1} ---")
    print(page.page_content)
    print("\n" + "=" * 80 + "\n")

✅ Arabic text has been reshaped, reordered, and stored back into docs.

--- Page 1 ---
ﺍﻟﻼﺋﺤﺔ ﺍﻟﺪﺍﺧﻠﻴﺔ 
ﻟﻜﻠﻴﺔ ﺍﻟﺤﺎﺳﺒﺎﺕ ﻭﺍﻟﺬﻛﺎﺀ ﺍﻻﺻﻄﻨﺎﻋﻲ 
Faculty of Computers and 
 Artificial Intelligence
(ﺑﻨﻈﺎﻡ ﺍﻟﺴﺎﻋﺎﺕ ﺍﻟﻤﻌﺘﻤﺪﺓ 
 
ﺟﺎﻣﻌﺔ ﻣﺪﻳﻨﺔ ﺍﻟﺴﺎﺩﺍﺕ 
 
ﺃﻛﺘﻮﺑﺮ 2019


--- Page 2 ---
ﺟﺎﻣﻌﺔ ﻣﺪﻳﻨﺔ ﺍﻟﺴﺎﺩﺍﺕ 
ﻛﻠﻴﺔ ﺍﻟﺤﺎﺳﺒﺎﺕ ﻭﺍﻟﺬﻛﺎﺀ ﺍﻻﺻﻄﻨﺎﻋﻲ 
  
Page 1 of 73        ﺍﻟﻼﺋﺤﺔ ﺍﻟﺪﺍﺧﻠﻴﺔ ﻟﻜﻠﻴﺔ ﺍﻟﺤﺎﺳﺒﺎﺕ ﻭﺍﻟﺬﻛﺎﺀ ﺍﻻﺻﻄﻨﺎﻋﻲ ﺑﻨﻈﺎﻡ ﺍﻟﺴﺎﻋﺎﺕ ﺍﻟﻤﻌﺘﻤﺪﺓ  2019          
ﻗﺎﺋﻤﺔ ﺍﻟﻤﺤﺘﻮﻳﺎﺕ 
ﻣﺴﻠﺴﻞ  ﺍﻟﻤﻮﺿﻮﻉ  ﺭﻗﻢ ﺍﻟﺼﻔﺤﺔ  
1  ﺗﻤﻬﻴﺪ 2 
2  ﺭﺅﻳﺔ ﻭﺭﺳﺎﻟﺔ ﻭﺃﻫﺪﺍﻑ ﺍﻟﻜﻠﻴﺔ   3 
3  ﻣﺎﺩﺓ1: ﻗﻮﺍﻋﺪ ﺍﻟﻘﺒﻮﻝ ﺑﺎﻟﻜﻠﻴﺔ 4 
4  ﻣﺎﺩﺓ2: ﺃﻗﺴﺎﻡ ﺍﻟﻜﻠﻴﺔ ﻭﺍﻟﺪﺭﺟﺎﺕ ﺍﻟﻌﻠﻤﻴﺔ 4 
5  ﻣﺎﺩﺓ3: ﻟﻐﺔ ﺍﻟﺘﺪﺭﻳﺲ 5 
6  ﻣﺎﺩﺓ4: ﻧﻈﺎﻡ ﺍﻟﺴﺎﻋﺎﺗﺎﻟﻤﻌﺘﻤﺪﺓ 5 
7  ﻣﺎﺩﺓ5: ﺍﻹﺭﺷﺎﺩ ﺍﻷﻛﺎﺩﻳﻤﻰ 6 
8  ﻣﺎﺩﺓ6: ﺍﻟﺘﺴﺠﻴﻞ ﻭﺍﻟﺤﺬﻑ ﻭﺍﻹﺿﺎﻓﺔ 7 
9  ﻣﺎﺩﺓ7: ﺍﻻﻧﺴﺤﺎﺏ ﻣﻦ ﺍﻟﻤﻘﺮﺭ 8 
10  ﻣﺎﺩﺓ8 :ﻗﻮﺍﻋﺪﺍﻟﻤﻮﺍﻇﺒﺔ ﻭﺍﻟﻐﻴﺎﺏ 8 
11  ﻣﺎﺩﺓ9: ﺍﻻﻧﺘﻘﺎﻝ ﺑﻴﻦ ﺍﻟﻤﺴﺘﻮﻳﺎﺕ 9 
12  ﻣﺎﺩﺓ10: ﺍﻹﻧﻘﻄﺎﻉ ﻋﻦ ﺍﻟﺪﺭﺍﺳﺔ 9 
13  ﻣﺎﺩﺓ11: ﺍﻟﻔﺼﻞ ﻣﻦ ﺍﻟﻜﻠﻴﺔ 9 
14  ﻣﺎﺩﺓ12 :ﺍﻟﺘﺤﻮﻳﻞ ﻭﻧﻘﻞ ﺍﻟﻘﻴﺪ ﻣﻦ ﺍﻟﻜﻠﻴﺎﺕ ﺍﻷﺧﺮﻯ 10 
15  ﻣﺎﺩﺓ13: ﻧﻈﺎﻡ ﺍﻹﻣﺘﺤﺎﻧﺎﺕ 10 
16  ﻣﺎﺩﺓ14: ﻧﻈﺎﻡ ﺍﻟﺘﻘﻮﻳﻢ 1

### Initialize Google Generative AI (Gemini)

Configuring the `gemini-2.5-flash` model via LangChain with zero temperature for deterministic responses, retry logic, and integration with the provided Google API key from Colab secrets.

In [9]:
google_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    max_tokens=None, # Max output tokens
    timeout=None,
    max_retries=2,
    convert_system_message_to_human=True,
    google_api_key=userdata.get('GOOGLE_API_KEY')
)

### Split Documents into Chunks

Using `RecursiveCharacterTextSplitter` to break the cleaned Arabic documents into manageable chunks (2000 characters with 200-character overlap) while respecting natural text boundaries. This prepares the text for effective embedding and retrieval. A sample chunk is displayed for verification.

In [11]:
# ⚙️ Configure the text splitter for balanced chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,          # Maximum number of characters per chunk
    chunk_overlap=1000,        # Overlap to preserve context between chunks
    separators=["\n\n", "\n", ".", " "]  # Priority of text break points
)

# ✂️ Split the loaded and fixed documents into smaller chunks
chunks = splitter.split_documents(docs)

print(f"✅ Successfully created {len(chunks)} chunks.\n")

# 🧾 Optional: Preview a sample chunk
sample = chunks[0]
print("Sample Chunk Metadata:", sample.metadata)
print("Sample Chunk Content:\n", sample.page_content)

✅ Successfully created 81 chunks.

Sample Chunk Metadata: {'producer': 'Microsoft® Word لبرنامج Office 365', 'creator': 'Microsoft® Word لبرنامج Office 365', 'creationdate': '2019-10-27T13:59:27+02:00', 'author': 'Magda Abou El Safa', 'moddate': '2019-10-27T14:01:24+02:00', 'title': 'كلية الحاسبات والذكاء الاصطناعى FCaI-USC', 'source': '/kaggle/input/fcai-usc-internal-regulations/2019.pdf', 'total_pages': 74, 'page': 0, 'page_label': '1'}
Sample Chunk Content:
 ﺍﻟﻼﺋﺤﺔ ﺍﻟﺪﺍﺧﻠﻴﺔ 
ﻟﻜﻠﻴﺔ ﺍﻟﺤﺎﺳﺒﺎﺕ ﻭﺍﻟﺬﻛﺎﺀ ﺍﻻﺻﻄﻨﺎﻋﻲ 
Faculty of Computers and 
 Artificial Intelligence
(ﺑﻨﻈﺎﻡ ﺍﻟﺴﺎﻋﺎﺕ ﺍﻟﻤﻌﺘﻤﺪﺓ 
 
ﺟﺎﻣﻌﺔ ﻣﺪﻳﻨﺔ ﺍﻟﺴﺎﺩﺍﺕ 
 
ﺃﻛﺘﻮﺑﺮ 2019


### 🌍 Bilingual Document Augmentation

Enhancing each document chunk by appending an English translation (using Google Translate) to the original Arabic text. This creates bilingual context for the LLM, improving cross-lingual retrieval and enabling more robust answers to questions in either language.

In [12]:
from deep_translator import GoogleTranslator

for doc in chunks:
    english = GoogleTranslator(source='auto', target='en').translate(doc.page_content)
    doc.page_content += "\n\n" + english

### Initialize Multilingual Embedding Model (E5 Base)

Switching to the `intfloat/multilingual-e5-base` embedding model—a high-performing multilingual encoder that supports Arabic, English, and many other languages. Configured to run on GPU (`cuda`) with normalized embeddings to enhance retrieval quality in semantic search over the FCAI regulations document.

In [13]:
# Multilingual model (supports Arabic, English, etc.)
model_name = "intfloat/multilingual-e5-base"

# Configuration for GPU and embedding normalization
model_kwargs = {'device': 'cpu'}  # Use 'cuda' for GPU; fallback to 'cpu' if no GPU available
encode_kwargs = {'normalize_embeddings': True}  # Normalization improves semantic search accuracy

# Initialize multilingual embedding model
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print(f"Loaded multilingual embedding model: {model_name} (GPU enabled)")


Loaded multilingual embedding model: intfloat/multilingual-e5-base (GPU enabled)


### Create and Persist Chroma Vector Store

Building a local Chroma vector database from the document chunks using multilingual embeddings. The vector store is persisted to disk (`./chroma_fcai_regulations_db`) for future reuse—enabling efficient semantic search over the FCAI internal regulations without re-embedding. Total embedded chunks: `{len(chunks)}`.

In [14]:
# Create a local Chroma vector store for the FCAI internal regulations document
vectorstore = Chroma.from_documents(
    documents=chunks,                               # The list of split document chunks
    embedding=hf,                                   # HuggingFaceEmbeddings instance
    persist_directory="./chroma_fcai_regulations_db"  # Local storage directory
)

# Persist the embeddings to disk
vectorstore.persist()

print("Chroma vector store for FCAI internal regulations created and saved successfully!")
print(f"Location: ./chroma_fcai_regulations_db")
print(f"Total embedded chunks: {len(chunks)}\n")

# To reload the vector store later (without re-embedding):
# vectorstore = Chroma(
#     persist_directory="./chroma_fcai_regulations_db",
#     embedding_function=hf
# )

# Optional: Display confirmation
print("Vector store instance:\n", vectorstore)


Chroma vector store for FCAI internal regulations created and saved successfully!
Location: ./chroma_fcai_regulations_db
Total embedded chunks: 81

Vector store instance:


### Custom Multilingual Prompt Template for RAG

Defined a bilingual (Arabic/English) prompt template that instructs the LLM to act as a knowledgeable assistant on the FCAI internal regulations. The prompt emphasizes context-based reasoning, inference with transparency, language consistency, and professional tone—ensuring accurate, detailed, and well-grounded responses in both Arabic and English.

In [15]:
prompt_template = """
أنت مساعد ذكي ومتعمق في التحليل، متخصص في الإجابة على الأسئلة المتعلقة باللائحة الداخلية لكلية الحاسبات والذكاء الاصطناعي.

You are an intelligent and thoughtful assistant specialized in answering questions about the Faculty of Computer and Artificial Intelligence Internal Regulations.

**Instructions:**
1. Analyze the provided context carefully and extract key information.
2. If the answer is not stated explicitly, infer it logically **based on the meaning and structure** of the context.
3. Combine information from multiple parts of the context when necessary to form a coherent, detailed answer.
4. Use reasoning and general understanding of academic or regulatory structures to fill in small gaps **only when it makes sense**.
5. Always clarify if a part of your answer is inferred or partially uncertain (for example: "يبدو من السياق أن ..." / "It can be inferred that ...").
6. Be slightly verbose — provide clear explanations or examples when helpful.
7. Maintain a professional and informative tone.
8. Answer in the same language as the question (Arabic or English).
9. Avoid adding any external information unrelated to the provided context.

**Context from Regulations:**
{context}

**Question:** {question}

**Answer (reasoned and detailed):**
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

### Set Up Retrieval-Augmented Generation (RAG) Pipeline

Configured a similarity-based retriever from the Chroma vector store (top 6 relevant chunks) and integrated it with the Gemini LLM via a `RetrievalQA` chain. The `"stuff"` chain type combines all retrieved context into a single prompt, and source documents are returned for transparency and verification.

In [16]:
# Create retriever interface from your persisted Chroma vector store


# retriever = vectorstore.as_retriever(
#     search_type="similarity",     # or "mmr" (diverse search)
#     search_kwargs={"k": 6}  # fetch_k is how many candidates to consider for mmr or similarity
# )

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 8,  # Reduced from 10 for more focused context
        "fetch_k": 25,  # Increased for better initial candidates
        "lambda_mult": 0.7  # Higher diversity
    }
)


# Create a RetrievalQA chain (LLM + Retriever)
qa_chain = RetrievalQA.from_chain_type(
    llm=google_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    chain_type="stuff",               # Concatenate all chunks into one prompt
    return_source_documents=True      # Include retrieved chunks in results
)

In [17]:
print(qa_chain.input_keys)

['query']


### 🧪 Evaluate RAG Chatbot on FCAI Regulations

Testing the Retrieval-Augmented Generation (RAG) system with a mix of Arabic and English questions about the Faculty of Computers and Artificial Intelligence (FCAI) internal regulations. The system retrieves relevant document chunks and generates answers using the Gemini LLM, while citing source pages for verification.

In [18]:
# 🔍 Arabic RAG Chatbot Test – FCAI Internal Regulations

questions = [
    "ما هي التخصصات الأربعة المتاحة في كلية الحاسبات والذكاء الاصطناعي وفقًا للائحة الداخلية؟",
    "what are the faculty departments ?",
    "كم عدد الساعات المعتمدة الإجمالية المطلوبة للتخرج من برنامج بكالوريوس علوم الحاسب؟",
    "ما هو الحد الأدنى للمعدل التراكمي (CGPA) المطلوب للبقاء في الكلية وفقًا للائحة؟",
    "What is the percentage allocated for the final exam in a regular course according to the evaluation system?",
    "What is the minimum cumulative GPA (CGPA) required to remain in the faculty according to the regulations?",
    "عاوز اسجل ماده software engineering 2 اي المطلوب عشان اخدها او اي المواد الل المفروض اخدها قبلها في قسم نظم المعلومات"
]

print("\n🧠 FCAI Internal Regulations RAG Evaluation\n")

for i, q in enumerate(questions, 1):
    print(f"🔹 Q{i}: {q}")
    result = qa_chain.invoke({"query": q})
    print(f"🧩 الإجابة: {result['result']}\n📚 المصادر:")
    for doc in result["source_documents"]:
        print(f"  - {doc.metadata.get('source', 'Unknown')} | صفحة {doc.metadata.get('page', 'N/A')}")
    print("=" * 80)
    time.sleep(2)

print("✅ Evaluation completed.")


🧠 FCAI Internal Regulations RAG Evaluation

🔹 Q1: ما هي التخصصات الأربعة المتاحة في كلية الحاسبات والذكاء الاصطناعي وفقًا للائحة الداخلية؟
🧩 الإجابة: وفقًا للائحة الداخلية لكلية الحاسبات والذكاء الاصطناعي، تمنح جامعة مدينة السادات درجة البكالوريوس في الحاسبات والذكاء الاصطناعي في أربعة تخصصات رئيسية. هذه التخصصات هي:

1.  **علوم الحاسب (Computer Science):** يُقدم هذا التخصص ضمن البرامج الأساسية للكلية.
2.  **نظم المعلومات (Information Systems):** يُقدم هذا التخصص أيضًا ضمن البرامج الأساسية للكلية.
3.  **الذكاء الاصطناعي (Artificial Intelligence):** يُقدم هذا التخصص ضمن البرامج الجديدة بمصروفات.
4.  **المعلوماتية الحيوية (Bioinformatics):** يُقدم هذا التخصص ضمن البرامج الجديدة بمصروفات.

وقد ورد ذلك صراحة في المادة المتعلقة بمنح درجة البكالوريوس، حيث تذكر اللائحة أن الجامعة تمنح درجة البكالوريوس في الحاسبات والذكاء الاصطناعي "تخصص علوم الحاساب، نظم المعلومات، الذكاء الاصطناعي، والمعلوماتية الحيوية من خلال البرامج التالية".
📚 المصادر:
  - /kaggle/input/fcai-usc-internal-regulations/2019